# Cleaning - Simply Recipes Ingredients List

Sources: 
* https://htmlpreview.github.io/?https://github.com/kulsoom-abdullah/kulsoom-abdullah.github.io/blob/master/AWS-lambda-implementation/model_implementation/recipe%20binary%20classification/recipe%20binary%20classification.html#Words-to-Remove
* https://towardsdatascience.com/https-towardsdatascience-com-end-to-end-recipe-cuisine-classification-e97f4ac22104


NYT - Natural Language Processor for Ingredients 

* https://github.com/nytimes/ingredient-phrase-tagger
* https://homepages.inf.ed.ac.uk/csutton/publications/crftutv2.pdf

CRF Tutorial 
* https://www.analyticsvidhya.com/blog/2018/08/nlp-guide-conditional-random-fields-text-classification/
* http://acepor.github.io/2017/03/06/CRF-Python/
* http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/
* https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html

In [1]:
import pandas as pd
import numpy as np
import re
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/markishab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
recipes_df = pd.read_pickle('../../data/02_intermediate/recipies_sr_inter.pkl')

In [3]:
recipes_df.reset_index(inplace=True, drop=True)

In [4]:
recipes_df.sort_values(by=['ingredients'])

,title,prep_time,cook_time,tags,ingredients,recipe_yield,byline,link_food
1254,Can You Make Cheesecake Without A Water Bath?,NaN,NaN,"['Tips', 'Cheesecake', 'Dessert']",[],NaN,Sara Bir,https://www.simplyrecipes.com/can_you_make_che...
190,What’s the Difference Between Stock and Broth?,NaN,NaN,"['Ingredient Guides', 'Tips', 'Chicken', 'Chic...",[],NaN,Emma Christensen,https://www.simplyrecipes.com/whats_the_differ...
1102,NaN,NaN,NaN,"['Produce', 'Recipe Collections', 'Baking', 'B...",[],NaN,'Sponsored Content,https://www.simplyrecipes.com/collection/5_way...
1680,How to Make Hard Boiled Eggs: 3 Fool-Proof Ways,NaN,NaN,"['Recipe Collections', 'Tips', 'Egg', 'Hard Bo...",[],NaN,Carrie Havranek,https://www.simplyrecipes.com/3_foolproof_ways...
1681,10 Ideas for the Best-Ever Eggs Benedict,NaN,NaN,"['Tips', 'Egg', 'Poached Eggs', 'Breakfast and...",[],NaN,Katie Workman,https://www.simplyrecipes.com/10_ideas_for_the...
167,NaN,NaN,NaN,"['Recipe Collections', 'Beef', 'Chicken', 'Cur...",[],NaN,Summer Miller,https://www.simplyrecipes.com/collection/10_sl...
163,NaN,NaN,NaN,"['Chicken', 'Chicken Tender', 'Vegetables', 'D...",[],NaN,Summer Miller,https://www.simplyrecipes.com/collection/4_ski...
158,How to Truss a Chicken,NaN,NaN,"['Tips', 'Chicken', 'Turkey', 'How To']",[],NaN,Carrie Havranek,https://www.simplyrecipes.com/how_to_truss_a_c...
1076,NaN,NaN,NaN,"['Recipe Collections', 'Cookie', 'Dessert', 'B...",[],NaN,Carrie Havranek,https://www.simplyrecipes.com/collection/best_...
140,NaN,NaN,NaN,"['Recipe Collections', 'Chicken', 'Dinner', 'G...",[],NaN,Claudia Cash,https://www.simplyrecipes.com/collection/12_am...


Let's drop the N/A columns

In [5]:
recipes_df.dropna(inplace=True)

## let's clean the list so that it can be used in a crf model 

In [6]:
ingredients_lists = list(recipes_df.ingredients)

In [7]:
len(ingredients_lists)

1177

In [8]:
ingredients_lists[0]

['8 slices sourdough bread',
 '4 tablespoon unsalted butter, at room temperature',
 '8 ounces (2 cups) shredded cheddar cheese',
 '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
 '8 to 12 slices ',
 'cooked bacon',
 '12 leaves butterhead or other crispy lettuce',
 'Kosher salt and black pepper']

In [9]:
ingredient_list_new = []
for recipe in ingredients_lists:
    sub_list = []
    for ingredient in recipe:
        sub_list.append(ingredient)
    ingredient_list_new.append(sub_list)

In [10]:
# ingredient_list_new = []
# for recipe in ingredients_lists:
#     sub_list = []
#     for ingredient in recipe:
#         ingredient_list_new.append(ingredient.split(" "))


In [11]:
ingredient_list_new

[['8 slices sourdough bread',
  '4 tablespoon unsalted butter, at room temperature',
  '8 ounces (2 cups) shredded cheddar cheese',
  '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
  '8 to 12 slices ',
  'cooked bacon',
  '12 leaves butterhead or other crispy lettuce',
  'Kosher salt and black pepper'],
 ['For the sauce:',
  '1 large onion, chopped',
  '6 garlic cloves, peeled',
  '1 pickled jalapeño pepper, seeded and chopped',
  '2 teaspoons Chipotle chile powder',
  '1 tablespoon tomato paste',
  '2 tablespoon Dijon mustard',
  '3/4 cup distilled white vinegar',
  '1 teaspoon paprika',
  '1/3 cup ketchup',
  '2 teaspoons Worcestershire sauce',
  '1/4 cup light brown sugar',
  'Salt',
  'For the pork:',
  '1 bay leaf',
  '3 pounds pork butt shoulder roast',
  'Hamburger buns'],
 ['12 strips bacon',
  '1/2 teaspoon ground black pepper (optional',
  ' Special equipment:',
  'Wire baking rack',
  '                                        

Let's isolate the actual ingredients

In [12]:
for element in ingredient_list_new:
    try:
        while(" " in element) : 
            element.remove(" ")
    except:
        pass

Replace ```'\\\\xa0'``` with space

In [13]:
new_list = []
for ingr_list in ingredient_list_new:
    ingr_list = [word.replace('\\\\xa0', ' ') for word in ingr_list]
    new_list.append(ingr_list)

In [14]:
len(new_list)

1177

In [15]:
final_list = []
for il in new_list:
    il = [word.replace('\\xa0', ' ') for word in il]
    final_list.append(il)

In [16]:
len(final_list)

1177

Let's figure out the final list

In [17]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            recipe.pop(idx)

In [18]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            print(ingredient)

In [19]:
len(final_list)

1177

In [20]:
recipes_df.ingredients[0]

['8 slices sourdough bread',
 '4 tablespoon unsalted butter, at room temperature',
 '8 ounces (2 cups) shredded cheddar cheese',
 '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
 '8 to 12 slices ',
 'cooked bacon',
 '12 leaves butterhead or other crispy lettuce',
 'Kosher salt and black pepper']

In [21]:
final_list[0]

['8 slices sourdough bread',
 '4 tablespoon unsalted butter, at room temperature',
 '8 ounces (2 cups) shredded cheddar cheese',
 '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
 '8 to 12 slices ',
 'cooked bacon',
 '12 leaves butterhead or other crispy lettuce',
 'Kosher salt and black pepper']

Make titles into list 

In [22]:
food_titles = list(recipes_df.title)

In [23]:
recipe_dict = dict()
for title, recipes in zip(food_titles, final_list):
#     print(title)
    recipe_dict[title] = recipes

In [24]:
len(recipes_df)

1177

In [25]:
recipes_df

,title,prep_time,cook_time,tags,ingredients,recipe_yield,byline,link_food
0,Grilled Cheese BLT,10 minutes,10 minutes,"['Dinner', 'Lunch', 'Sandwich', 'Favorite Summ...","[8 slices sourdough bread, 4 tablespoon unsalt...",4 sandwiches,Aaron Hutcherson,https://www.simplyrecipes.com/recipes/grilled_...
1,Pulled Pork Sandwich,10 minutes,"2 hours, 45 minutes","['Dinner', 'Sandwich', 'Budget', 'Comfort Food...","[For the sauce:, 1 large onion, chopped, 6 gar...",Serves 6 to 8,Elise Bauer,https://www.simplyrecipes.com/recipes/pulled_p...
2,How to Make Bacon in the Oven,5 minutes,20 minutes,"['Tips', 'Breakfast and Brunch', 'Baking', 'Ho...","[12 strips bacon, 1/2 teaspoon ground black pe...",12 strips,Nick Evans,https://www.simplyrecipes.com/recipes/how_to_m...
3,Sausage Stuffed Zucchini,15 minutes,1 hour,"['Dinner', 'Favorite Summer', 'Make-ahead', 'I...","[2 tablespoons extra virgin olive oil, 1/2 pou...",Serves 4,Elise Bauer,https://www.simplyrecipes.com/recipes/italian_...
5,Ginger Pork Rice Bowls,15 minutes,20 minutes,"['Eat Your Food', 'Family Dinner Ideas', 'Dinn...","[For the bowls:, 1 tablespoon olive oil, 1/2 c...",4-6 servings,Nick Evans,https://www.simplyrecipes.com/recipes/ginger_p...
6,Hawaiian SPAM Tacos with Pineapple,10 minutes,30 minutes,"['Eat Your Food', 'Dinner', 'Kid-friendly', 'H...","[DAD ADD: Haba, ñero Guacamole, 1 habañero pep...","12 small tacos, serving 4 adults",Nick Evans,https://www.simplyrecipes.com/recipes/hawaiian...
7,Air Fryer Chinese Egg Rolls,20 minutes,25 minutes,"['Appetizer', 'Snack', 'Air Fryer', 'Quick and...","[For the egg rolls:, 1 tablespoon olive oil, 1...",12 egg rolls,Nick Evans,https://www.simplyrecipes.com/recipes/air_frye...
8,Grilled Bacon-Wrapped Stuffed Hot Dogs,10 minutes,11 minutes,"['Dinner', 'Favorite Summer', 'Grill', 'Bacon'...","[1 teaspoon ketchup, 1 teaspoon Dijon mustard,...",Makes 4 stuffed hot dogs,Elise Bauer,https://www.simplyrecipes.com/recipes/grilled_...
9,Italian Grilled Cheese Sandwiches,10 minutes,20 minutes,"['Eat Your Food', 'Dinner', 'Sandwich', 'Kid-f...","[For the DAD ADD: , Olive and Cauliflower Giar...",4 sandwiches,Nick Evans,https://www.simplyrecipes.com/recipes/italian_...
11,Biscuits and Gravy Casserole,25 minutes,35 minutes,"['Breakfast and Brunch', 'Casserole', 'Comfort...","[For the biscuits:, 2 1/4 cups (315 g) all-pur...",12 servings,Irvin Lin,https://www.simplyrecipes.com/recipes/biscuits...


In [26]:
def clumpFractions(s):
    """
    Replaces the whitespace between the integer and fractional part of a quantity
    with a dollar sign, so it's interpreted as a single token. The rest of the
    string is left alone.

        clumpFractions("aaa 1 2/3 bbb")
        # => "aaa 1$2/3 bbb"
    """
    return re.sub(r'(\d+)\s+(\d)/(\d)', r'\1$\2/\3', s)

In [27]:
def choose_american_unit(s):
    """
    Tokenize on parenthesis, punctuation, spaces and American units followed by a slash.

    We sometimes give American units and metric units for baking recipes. For example:
        * 2 tablespoons/30 mililiters milk or cream
        * 2 1/2 cups/300 grams all-purpose flour

    The recipe database only allows for one unit, and we want to use the American one.
    But we must split the text on "cups/" etc. in order to pick it up.
    """

    american_units = ['cup', 'tablespoon', 'teaspoon', 'pound', 'ounce', 'quart', 'pint']
    for unit in american_units:
        s = s.replace(unit + '/', unit + ' ')
        s = s.replace(unit + 's/', unit + 's ')

    return clumpFractions(s)

In [28]:
final_list_new = []
for recipe in final_list:
    sub_list = []
    for ingredient in recipe:
        word = ingredient.replace("(", "")
        word2 = choose_american_unit(word)
        sub_list.append(word2.replace(")", ""))
    final_list_new.append(sub_list)

In [29]:
final_list_new

[['8 slices sourdough bread',
  '4 tablespoon unsalted butter, at room temperature',
  '8 ounces 2 cups shredded cheddar cheese',
  '2 slicing tomatoes such as beefsteak, Brandywine, or Cherokee purple, sliced 1/4-inch thick',
  '8 to 12 slices ',
  'cooked bacon',
  '12 leaves butterhead or other crispy lettuce',
  'Kosher salt and black pepper'],
 ['1 large onion, chopped',
  '6 garlic cloves, peeled',
  '1 pickled jalapeño pepper, seeded and chopped',
  '2 teaspoons Chipotle chile powder',
  '1 tablespoon tomato paste',
  '2 tablespoon Dijon mustard',
  '3/4 cup distilled white vinegar',
  '1 teaspoon paprika',
  '1/3 cup ketchup',
  '2 teaspoons Worcestershire sauce',
  '1/4 cup light brown sugar',
  'Salt',
  '1 bay leaf',
  '3 pounds pork butt shoulder roast',
  'Hamburger buns'],
 ['12 strips bacon',
  '1/2 teaspoon ground black pepper optional',
  ' Special equipment:',
  'Wire baking rack',
  '                                              '],
 ['2 tablespoons extra virgin oliv

In [30]:
len(final_list)

1177

In [31]:
len(final_list_new)

1177

In [32]:
recipes_df['ingredients'] = final_list_new

In [35]:
recipes_df.to_pickle('../../data/02_intermediate/recipes_sr_final.pickle')